In [1]:
import os
import cv2
import sys
import json

from tqdm import tqdm
import pandas as pd

import ultralytics

from ultralytics import YOLO
from matplotlib import pyplot as plt

from glob import glob
import numpy as np
from mapcalc import calculate_map, calculate_map_range

ultralytics.checks()

Ultralytics YOLOv8.0.114  Python-3.9.16 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU, 16384MiB)
Setup complete  (20 CPUs, 31.7 GB RAM, 811.4/934.5 GB disk)


In [2]:
def draw_boxes(image_path, boxes, labels, scores, ground_truth_boxes, ground_truth_labels, output_path, normalized_bbox=True):
    # Read the image
    image = cv2.imread(image_path)

    # Get the image dimensions
    height, width = image.shape[:2]

    # Iterate over the boxes
    for (box, label, score) in zip(boxes, labels, scores):
        # Convert normalized coordinates to pixel coordinates
        x1, y1, x2, y2 = tuple(box)
        if normalized_bbox:
            x1, y1, x2, y2 = int(x1 * width), int(y1 * height), int(x2 * width), int(y2 * height)
        else:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Draw the box

        # Multi-class
        if label == 0:
            tumor = "benign"
            color = (0, 255, 0)
        elif label == 1:
            tumor = "malignant"
            color = (0, 0, 255)
        
        # # Single-class
        # if label == 0:
        #     tumor = "tumor"
        #     color = (255, 0, 0)

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Prepare the label and confidence
        text = f"{tumor}: {score:.2f}"

        # Draw the label and confidence
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    for (box, label) in zip(ground_truth_boxes, ground_truth_labels):
        # Convert normalized coordinates to pixel coordinates
        x1, y1, x2, y2 = tuple(box)
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Draw the box

        if label == 0:
            tumor = "I-benign"
            color = (0, 255, 0)
        elif label == 1:
            tumor = "I-malignant"
            color = (0, 0, 255)

        # Draw the label and confidence
        cv2.putText(image, tumor, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

        print(image_path)
        print(label)

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)

    # Save the image
    cv2.imwrite(output_path, image)

In [3]:
# Best model for multi-class classification
MODEL_PATH = "C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/AIPlus_BC_Practicum-main/AIPlus_BC_Practicum-main/yolov8/runs/detect/multi_cls_wholeset/yolo8l_0.00001/weights/best.pt"
TEST_SET_PATH = "C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/"

In [4]:
imgs_dir = os.path.join(TEST_SET_PATH, "imgs")
output_dir = os.path.join(TEST_SET_PATH, "outputs")
jsons_dir= os.path.join(TEST_SET_PATH, "jsons")
img_list = glob(os.path.join(imgs_dir, "*.jpg"))

print(f"Total Number of Test Images: {len(img_list)}")

In [7]:
# Load Trained Model
model = YOLO(MODEL_PATH)

## Predict

In [8]:

map_dict = {"img_name":[], "mAP50":[], "mAP50-95":[]}

for img in tqdm(img_list):

    result = model.predict(img, imgsz=640, conf=0.5, iou=0.7, augment=True)
    # result = model.predict(img, imgsz=640, conf=0.25, iou=0.7, augment=True)

    img_name = os.path.basename(img)
    output_path = os.path.join(output_dir, img_name)
    json_file = os.path.join(jsons_dir, img_name.replace(".jpg", ".json"))

    with open(json_file, "r") as f:
        test_data = json.load(f)

    ground_truth = {"boxes":[], "labels":[]}
    result_dict = {"boxes":[], "labels":[], "scores":[]}

    for shape in test_data["shapes"]:
        # if shape['label'] != "border":
        #     ground_truth["labels"].append(shape["label"])
        #     ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])   

        # Multi-class
        if shape['label'] == "benign":
            shape['label'] = 0
            ground_truth["labels"].append(shape["label"])
            ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])            
        elif shape['label'] == "malignant":
            shape['label'] = 1
            ground_truth["labels"].append(shape["label"])
            ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])
        
        # if shape['label'] != 'border':
        #     shape['label'] = 0
        #     ground_truth["labels"].append(shape["label"])
        #     ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])

    cls_list = []
    conf_list = []
    nboxes_list = []
    boxes_list = []

    for i in range(len(result[0].boxes.cls.cpu().detach().numpy())):
        cls_list.append(int(result[0].boxes.cls.cpu().detach().numpy()[i].tolist()))
        conf_list.append(result[0].boxes.conf.cpu().detach().numpy()[i].tolist())
        nboxes_list.append(result[0].boxes.xyxyn.cpu().detach().numpy()[i].tolist())
        boxes_list.append(result[0].boxes.xyxy.cpu().detach().numpy()[i].tolist())

        result_dict["labels"].append(int(result[0].boxes.cls.cpu().detach().numpy()[i].tolist()))
        result_dict["boxes"].append(result[0].boxes.xyxy.cpu().detach().numpy()[i].tolist())
        result_dict["scores"].append(result[0].boxes.conf.cpu().detach().numpy()[i].tolist())
    
    draw_boxes(img, nboxes_list, cls_list, conf_list, ground_truth['boxes'], ground_truth['labels'], output_path)

    # print(ground_truth)
    # print(result_dict)

    map50 = calculate_map(ground_truth, result_dict, 0.5)
    map5095 = calculate_map_range(ground_truth, result_dict, 0.5, 0.95, 0.05)

    map_dict["img_name"].append(img_name)
    map_dict["mAP50"].append(map50)
    map_dict["mAP50-95"].append(map5095)

  0%|          | 0/239 [00:00<?, ?it/s]
image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645110.jpg: 608x640 1 malignant, 321.2ms
Speed: 5.5ms preprocess, 321.2ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
  0%|          | 1/239 [00:03<15:23,  3.88s/it]


C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645110.jpg
1


image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645117.jpg: 640x640 1 malignant, 253.2ms
Speed: 3.0ms preprocess, 253.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
  1%|          | 2/239 [00:04<06:56,  1.76s/it]
image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645118.jpg: 640x640 1 malignant, 59.5ms
Speed: 3.0ms preprocess, 59.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645163.jpg: 640x640 1 malignant, 61.0ms
Speed: 3.0ms preprocess, 61.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
  2%|▏         | 4/239 [00:04<02:45,  1.42it/s]


C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645117.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645118.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\01E6C75C2D5E418DAB90F7562CA4083B_2645163.jpg
1


image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\04A195C7193647B599073C0FD2EF4D7D_2620119.jpg: 608x640 1 malignant, 59.9ms
Speed: 2.0ms preprocess, 59.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\04A195C7193647B599073C0FD2EF4D7D_2620171.jpg: 640x640 1 malignant, 57.2ms
Speed: 3.0ms preprocess, 57.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
  3%|▎         | 6/239 [00:04<01:34,  2.46it/s]


...

C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\CCF21CCF218E47BF9710E8F5B7ED299F_2667929.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\D8DEE6B5198D41C7A3BDE3BD5AB3A987_2757865.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\E6913F34515D46AEA59A3015AD70D71E_2800523.jpg
1


Speed: 2.0ms preprocess, 49.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
 98%|█████████▊| 235/239 [00:25<00:00, 13.69it/s]
image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\E73534BF231E40D3806A253D249E73B4_2846495.jpg: 512x640 1 malignant, 66.3ms
Speed: 2.5ms preprocess, 66.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\E9CC5D13DDEC4A0FAF45D9920D5E414F_2489528.jpg: 544x640 1 malignant, 54.0ms
Speed: 3.5ms preprocess, 54.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
 99%|█████████▉| 237/239 [00:25<00:00, 13.19it/s]


C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\E6913F34515D46AEA59A3015AD70D71E_2800563.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\E73534BF231E40D3806A253D249E73B4_2846495.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\E9CC5D13DDEC4A0FAF45D9920D5E414F_2489528.jpg
1


image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\E9CC5D13DDEC4A0FAF45D9920D5E414F_2489549.jpg: 544x640 1 malignant, 51.5ms
Speed: 4.2ms preprocess, 51.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\FrankW\Desktop\test\Code_Monkeys\yolov8\dataset\external_test_cropped\imgs\F5B281A3EFB54F188F4E20843DAF8262_2283978.jpg: 640x448 1 benign, 49.2ms
Speed: 2.0ms preprocess, 49.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
100%|██████████| 239/239 [00:25<00:00,  9.39it/s]

C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\E9CC5D13DDEC4A0FAF45D9920D5E414F_2489549.jpg
1
C:/Users/FrankW/Desktop/test/Code_Monkeys/yolov8/dataset/external_test_cropped/imgs\F5B281A3EFB54F188F4E20843DAF8262_2283978.jpg
0


In [9]:
df = pd.DataFrame(map_dict)

In [10]:
df

,img_name,mAP50,mAP50-95
0,01E6C75C2D5E418DAB90F7562CA4083B_2645110.jpg,1.0,0.6
1,01E6C75C2D5E418DAB90F7562CA4083B_2645117.jpg,1.0,0.5
2,01E6C75C2D5E418DAB90F7562CA4083B_2645118.jpg,1.0,0.6
3,01E6C75C2D5E418DAB90F7562CA4083B_2645163.jpg,1.0,0.5
4,04A195C7193647B599073C0FD2EF4D7D_2620119.jpg,1.0,0.9
...,...,...,...
234,E6913F34515D46AEA59A3015AD70D71E_2800563.jpg,0.0,0.0
235,E73534BF231E40D3806A253D249E73B4_2846495.jpg,1.0,0.4
236,E9CC5D13DDEC4A0FAF45D9920D5E414F_2489528.jpg,1.0,0.9
237,E9CC5D13DDEC4A0FAF45D9920D5E414F_2489549.jpg,1.0,0.9


In [11]:
# mAP50
df['mAP50'].mean()

0.9058577405857741

In [12]:
# Images Not detected Correctly
df[df['mAP50']==0]

,img_name,mAP50,mAP50-95
53,1.2.826.0.1.3680043.2.461.12524243.1573887775.jpg,0.0,0.0
97,1.2.826.0.1.3680043.2.461.12581554.4277809629.jpg,0.0,0.0
98,1.2.826.0.1.3680043.2.461.12583566.2974374378.jpg,0.0,0.0
99,1.2.826.0.1.3680043.2.461.12583569.1907021304.jpg,0.0,0.0
108,1.2.826.0.1.3680043.2.461.12595881.3853898007.jpg,0.0,0.0
111,1.2.826.0.1.3680043.2.461.12638760.3079405872.jpg,0.0,0.0
114,1.2.826.0.1.3680043.2.461.12638840.163889883.jpg,0.0,0.0
136,29C71F8FC50C4104B07BB55C96A8AA95_7704699.jpg,0.0,0.0
137,29C71F8FC50C4104B07BB55C96A8AA95_7704703.jpg,0.0,0.0
148,3B8AD6EB99C3477984402F7D633DF506_2073429.jpg,0.0,0.0


In [13]:
len(df[df['mAP50']==0])

21

In [14]:
# mAP50-95
df['mAP50-95'].mean()

0.4853556485355649

In [15]:
# Save mAP results to csv files
df.to_csv(os.path.join(TEST_SET_PATH, "mAP_results.csv"))